# Lab 9 - Arduino introduction and amplified photodiode
### PHYS 222, Spring 2024, Dr. Ben Zwickl, Dr. Ke Xu, Dr. Michael Pierce 

### Goals
* Connect to Arduino
* Controlling the on-board LED
* Controlling an LEDs with a digital output.
* Using the Arduino's `analogRead()` function as an oscilloscope
* Building an amplified photodiode using an LM358 op-amp
* Measuring the pulse patterns from an infrared remote control

**Note:** This lab does not require any Python coding. All the coding will be done in the Arduino IDE programming environment. 


In [2]:
# load packages at the top of your notebook. 
# We will always use numpy and matplotlib.pyplot.
# In future weeks, you should add these lines yourself

import numpy as np
import matplotlib.pyplot as plt

# Task 1: Control the on-board LED for Arduino (2 pts)

Use this tutorial connect to your Arduino https://www.arduino.cc/en/Guide/ArduinoUno. 
1. Make sure you get the Arduino Desktop IDE working (not just the Arduino Web Editor). The Desktop IDE has important features, especially the Serial Plotter.
1. Make sure you can run the **Example > 01.Basics > Blink** sketch (part of the guide).
1. How many "blinks per second" does this example program do?
1. Google the official Arduino documentation for the `delay` function. How many inputs does this function take? What do the inputs mean? Why does the example program have the line `delay(1000)`?
1. Modify the program to flash 4 times per second. 

**Copy your final Arduino code into your notebook**

# Task 2: Control an off-board LED using Arduino digital out (2 pts)
![Controlling LED with Pin 13](Fig_Schematic_for_Pin13_LED.PNG)
1. Build this circuit, connect it to the Arduino's pin 13 and the Arduino's ground.
1. To control the output of Pin 13, we will use the Arduino's digialWrite function to switch the output between HIGH and LOW. Use the example in the official documentation to see how to switch a pin between high and low. https://www.arduino.cc/reference/en/language/functions/digital-io/digitalwrite/
1. Use the M2K oscilloscope to measure both the voltage output of Pin 13 and the voltage across the LED. 
1. What is the current through the LED when it is ON? Why is the resistor important in this circuit? See Max Forward current rating according to datasheets for the [Blue LED](https://www.digikey.com/en/products/detail/cree-inc/C503B-BCN-CV0Z0462/6138534) and [Red LED](https://www.digikey.com/en/products/detail/cree-inc/C503B-RCN-CW0Z0AA2/6138549)



**Copy your final Arduino code into your notebook**

# Task 3: Use the Arduino to read and plot analog voltage signals (2 pts)

This is a "skill building" task that will be useful for more sophisticated measurements, such as the pulse sensor.

In addition to having pins for digital inputs and outputs, the Arduino has dedicated pins for analog Inputs. You can think of these pins as multimeters or oscilloscopes that can only sample at a low rate (10s or 100s of times per second). The main purpose of these pins is to read analog sensors. 

#### Sub-tasks


1. **Use the M2K's signal generator to create a slow oscillation** (1 Hz, 3 Vpp, 1.5 V offset). The goal is to have a signal that oscillates, but doesn't go negative because the analog input cannot read negative voltages. Connect the M2K's W1 output to the analog input pin A3 on the Arduino Uno.
1. **Read the [official documentation](https://www.arduino.cc/reference/en/language/functions/analog-io/analogread/)** on the analogRead() function.
1. **Use the Example** in the documentation for your Arduino sketch (program).
1. **Use serial plotter to visualize the signal.** Start the serial plotter under the menu **Tools > Serial Plotter**. A new window with a graph should pop up.
    * What happens to the negative values of the waveform? 
    * What are the range of x-values?
1. **Save a screenshot** of the serial plotter.

**Copy your final Arduino code into your notebook**

# Task 4: Convert Analog output values to millivolts (2 pts)

This task is an extension of Task 4 that describes more about the analog inputs:
* The Arduino's analog inputs only accept values between 0-5 V. Anything lower or higher is clipped.
* The analog to digital converter has 10 bits of resolution, so the range from 0-5 V gets divided into $2^{10} = 1024$ steps.
* The analog input stores the voltage as an integer between 0 (corresponding to 0 V) and 1023 (corresponding to 5V). 

To convert from analog input value (integer, $\textrm{val}$) into a voltage $V$ in millivolts, use:

$$ V = \frac{5.0 \textrm{ V}}{1024.} \times \textrm{val} \times 1000$$

* Note the use of `5.0` and `1024.` which ensure that `V` will be treated as a decimal number rather than an integer. You can also wrap the voltage with a `float(V)` which converts to a floating point number. 
* You may be wondering why we use mV rather than V. It is because the autoranging on the serial plotter works better with larger numbers. 

#### Sub-tasks

1. Explain the meaning of the equation in your own words.
1. Modify the Arduino code from Task 4: 
    * create a new `float` type variable named `mv` (for millivolts). 
    * after each `analogRead() of `val`, add a line where you calculate the value for `mv` in terms of `val`.
    * modify the `Serial.println()` statement to output the value in millivolts rather than the integer. 
1. Run the new code and visualize the output run serial plotter. 
1. Take a screenshot of the serial plotter output. You should now see the sine wave with a value in mV on the y-axis and it should match the settings in the M2K's signal generator. 


**Copy your final Arduino code into your notebook**

# Task 5: Build the amplified photodetector and measure the signal from an infrared (IR) remote control (2 pts)

The photodiode is a flat rectangular chip with two stubby legs. It should be in your bag with LEDs and diodes. You can identify the cathode leg by a very small "flag" that sticks out the side of the leg.
![Schematic for amplified photodiode](Photodiode_with_transimpedance_amp.jpg)

### LM 358 Op-amp

The circuit uses LM358 op-amps because they work well with a "single-sided" supply (the negative supply is at ground). This means the Arduino's fixed 5V output and ground pins can provide the op-amp supply voltages. Note the LM358 pin diagram refers to V+ and V- as the power supply inputs, rather than the non-inverting and inverting inputs. $V+ = 5 \textrm{ V}$ and $V- = 0 \textrm{ V}$ (ground). 

![Pin diagram for the LM358](LM358_pin_configuration.PNG)

The LM 358 has two op-amps integrated on a single chip. They use the same power supply, but have separate inputs and outputs. 
* IN1-, IN1+, OUT1 are for the 1st op-amp
* IN2-, IN2+, OUT2 are for the 2nd op-amp



### What is a transimpedance amplifier?

The first stage of the circuit is known as a transimpedance amplifier, which is a current to voltage converter. The photodiode produces a current in response to the amount of incident light. The transimpedance amplifier converts that to a proportional voltage.

## Subtasks

1. **Theory question** How much current does the photodiode need to produce for a 1V output from this transimpedance amplifier?
1. **Build the circuit.** Use IN1-, IN1+, and OUT1 for the build today. We do not need to use IN2+, IN2-, and OUT2.
1. **Quick testing:** If you block the light from reaching the photodiode, does the voltage drop? If you shine a flashlight onto the photodiode, does it saturate the amplifier output?
1. **Save and upload a screenshot from the Serial Plotter** as you vary the illumination on the photodiode. Prove it is working okay.
1. **Shine a remote control (from a TV or projector) at the photodiode. Measure the output voltage using the M2K oscilloscope.** You should be able to see well defined pulse sequences unique to each button press. If you switch between buttons, you should see the pattern change in small but repeatable ways. **Make sure to show a couple of screenshots in your lab notebook.** 

### Example of wiggling the phone above the photodiode
Measurement using the Arduino's analog input and the serial plotter. The y-axis has been converted to volts. 
![Variable output of amplified photodiode as a flashlight is wiggled around](Wiggling_phone_flashlight_above_photodiode.jpg)

### Example measurement of a remote control button press (as measured by the M2K oscilloscope)
This shows the output of a circa 2018 Samsung TV remote control when the channel down button is pressed. The pattern repeats every time the button is pressed. If you switch to channel up, a few parts of the pattern change. 
![Samsung Channel Down](Samsung_channel_down.jpg)


**Save screenshots into your lab notebook**